# Calculate predictors

- ERSST Predictors (6)
    * SIOD_E
    * SIOD_W
    * SST_Med
    * TSA
    * TNA
    * SST_mdr

- SATA Predictors (4)
    * lnh
    * lsh
    * onh
    * osh

- 2-D (year x month) textfiles (10)
    * SOI (2)
    * AMO
    * NAO
    * PDO
    * NP
    * Ninos (4)

#### Import packages

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

# Temporal Range for all indices:

Months: AMJ\
Years: 1901 - 2017

In [2]:
month_s = 4  # April
month_e = 6  # June
year_s = 1901
year_e = 2017

# ERSST Predictors (6)

### Define function to select subset, calculate mean, and save to new file

This function creates one seasonal value for each year that is regionally averaged.\
It uses raw data - not anomalies or detrended.

Can Handle Lon = 0:359 if area crosses 180 meridian\
Or Lon = -180:179 if are crosses 0 median

Missing values are already nan

In [3]:
def index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
):
    ersst = xr.open_dataset("da_my_ersst.nc").sst

    if np.sign(lon_s) == -1:  # test for input longitude sign and eventually reshape dataset
        i = ersst.lon
        ersst = xr.concat(
            [ersst.sel(lon=i[(i >= 180)]), ersst.sel(lon=i[(i < 180)])], dim="lon"
        )
        ersst = ersst.assign_coords({"lon": (((ersst.lon + 180) % 360) - 180)})
        print("data has been reshaped")

    # Now call data in given time and space range and compute means, then save data array to new file

    ersst = (
        ersst.sel(
            time=(ersst["time.month"] >= month_s)
            & (ersst["time.month"] <= month_e)
            & (ersst["time.year"] >= year_s)
            & (ersst["time.year"] <= year_e),
            lat=slice(lat_s, lat_e),
            lon=slice(lon_s, lon_e),
        )
        .groupby("time.year")
        .mean(("time", "lat", "lon"))
    )
    ersst.to_netcdf(f"da_pred_{name}.nc")
    return ersst

### SIOD_E - (Eastern Suntropical Indian Ocean)

In [4]:
name = "siod_e"
lat_s = -28
lat_e = -18
lon_s = 90
lon_e = 100

siod_e = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

### SIOD_W - (Western Subtropical Indian Ocean)

In [5]:
name = "siod_w"
lat_s = -37
lat_e = -27
lon_s = 55
lon_e = 65

siod_w = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

### SST_Med - (Mediterranean Sea)

In [6]:
name = "sst_med"
lat_s = 30
lat_e = 45
lon_s = 0
lon_e = 25

sst_med = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

### TSA - (Tropical South Atlantic)

In [7]:
name = "tsa"
lat_s = -20
lat_e = 0
lon_s = -30
lon_e = 10

tsa = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

data has been reshaped


### TNA - (Tropical North Atlantic)

In [8]:
name = "tna"
lat_s = 5
lat_e = 25
lon_s = -55
lon_e = -15

tna = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

data has been reshaped


### SST_MDR - (Hurricane main development region)

In [9]:
name = "sst_mdr"
lat_s = 10
lat_e = 20
lon_s = -85
lon_e = -20

tna = index_from_ersst(
    name, month_s, month_e, year_s, year_e, lat_s, lat_e, lon_s, lon_e
)

data has been reshaped


# SATA Predictors (4)
This data comes in global monthly means.\
Here according seasonal means are computed

Load the four sata files and merge them together in one xarray ds and set missing values to NAN

In [10]:
# function to laod csv files as pandas df and renames index and variable
def load_sata(name):
    a = (
        pd.read_csv(f"da_o_sata_{name}.csv", header=4)
        .rename(columns={"Value": f"sata_{name}", "Year": "time"})
        .set_index("time")
    )
    return a


# use function to load and concat the four time series
sata = pd.concat(
    [load_sata("lnh"), load_sata("lsh"), load_sata("onh"), load_sata("osh")], axis=1
).to_xarray()


#set missing values (-999) to nan
sata = sata.where(sata != -999)

Select temporal range, compute seasonal means and save data to new file

In [12]:
# assign daterange time coordinate for better handling
sata = sata.assign_coords(
    time=("time", pd.date_range("1880-1-1", "2020-10-1", freq="MS"))
)

# select time range, compute seasonal mean
sata = (
    sata.sel(
        time=(sata["time.month"] >= month_s)
        & (sata["time.month"] <= month_e)
        & (sata["time.year"] >= year_s)
        & (sata["time.year"] <= year_e),
    )
    .groupby("time.year")
    .mean("time")
)

# Save data to new file
sata.to_netcdf("da_pred_sata_all.nc")

# 2-D (year x month) text data (10)

- SOI (2)
- AMO
- NAO
- ninos (4)
- PDO
- NP

### For each data file:
1. Import Data to Xarray 

# SOI (Souther Oscillation Index) (2)
## Darwin and Tahiti SLP

Load the two text files into one Xarray Data Set and set coordinate axes:

In [13]:
soi = xr.Dataset(
    data_vars={
        "slp_darwin": (("year", "month"), np.loadtxt("da_o_soi_dar.txt")[:, 1:]),
        "slp_tahiti": (("year", "month"), np.loadtxt("da_o_soi_tah.txt")[:, 1:]),
    },
    coords={
        "year": np.arange(1855, 2021),
        "month": np.arange(1, 13),
    },
    attrs={"unit": "hPa"},
)

#set missing values (-990) to nan
soi = soi.where(soi != -990)

Select temporal range, compute seasonal means and save data to new file:

In [14]:
# select time range, compute seasonal mean
i = soi.month
ii = soi.year
soi = soi.sel(
    month=i[(i >= month_s) & (i <= month_e)],
    year=ii[(ii >= year_s) & (ii <= year_e)],
).mean('month')

# Save data to new file
soi.to_netcdf("da_pred_soi_all.nc")

# AMO (Atlantic Multidecadal Oscillation) (1)

Load the text files into one Xarray Data Set and set coordinate axes:

In [15]:
amo = xr.DataArray(
    data=np.loadtxt(open('da_o_amo_detrend.txt').readlines()[:-4], skiprows=1)[:,1:],  # use nested open to skip bottom lines string text, and first column (years)
    dims=("year", "month"),
    name="amo",
    coords={
        "year": np.loadtxt(open('da_o_amo_detrend.txt').readlines()[:-4], skiprows=1)[:,0],
        "month": np.arange(1, 13),
    },
    attrs={"unit": "degree C"},
)

# set non-values (-99.99) to NAN 
amo = amo.where(amo != -99.99)

Select temporal range, compute seasonal means and save data to new file:

In [16]:
# select time range, compute seasonal mean

i = amo.month
ii = amo.year

amo = amo.sel(
    month=i[(i >= month_s) & (i <= month_e)],
    year=ii[(ii >= year_s) & (ii <= year_e)],
).mean('month')

# Save data to new file
amo.to_netcdf("da_pred_amo.nc")

In [17]:
amo

<xarray.DataArray 'amo' (year: 117)>
array([ 0.16      , -0.15466667, -0.12166667, -0.41      , -0.13      ,
       -0.06033333, -0.184     , -0.14233333, -0.092     , -0.33533333,
       -0.299     , -0.14333333, -0.48466667, -0.34133333,  0.202     ,
       -0.024     , -0.23733333, -0.27933333, -0.148     , -0.29      ,
       -0.25433333, -0.353     , -0.34166667, -0.03733333, -0.15033333,
        0.099     ,  0.13566667, -0.00766667, -0.10833333, -0.136     ,
        0.21233333,  0.201     ,  0.24633333, -0.06      ,  0.09866667,
        0.18233333,  0.21333333,  0.199     ,  0.122     , -0.00933333,
        0.02366667,  0.22066667, -0.115     ,  0.315     ,  0.29566667,
        0.01433333, -0.13633333, -0.00833333,  0.02333333, -0.07866667,
        0.20833333,  0.24966667,  0.308     ,  0.04933333,  0.15266667,
       -0.089     , -0.072     ,  0.253     , -0.022     ,  0.23566667,
        0.147     ,  0.00866667, -0.014     , -0.03866667, -0.11266667,
       -0.00533333, -0.15966667, -0.186     ,  0.05633333, -0.076     ,
       -0.37366667, -0.43366667, -0.20866667, -0.496     , -0.34866667,
       -0.47166667, -0.17033333, -0.249     , -0.137     ,  0.07466667,
       -0.08033333, -0.18566667, -0.014     , -0.21566667, -0.28866667,
       -0.27166667,  0.09966667,  0.11566667, -0.11933333, -0.082     ,
       -0.14366667, -0.19166667, -0.17933333, -0.21666667,  0.22933333,
       -0.068     ,  0.01      ,  0.37966667,  0.12266667,  0.034     ,
        0.05      , -0.055     ,  0.136     ,  0.088     ,  0.295     ,
        0.27033333,  0.11033333,  0.148     , -0.033     ,  0.43466667,
        0.131     ,  0.17166667,  0.085     , -0.021     , -0.00933333,
        0.3       ,  0.29133333])
Coordinates:
  * year     (year) float64 1.901e+03 1.902e+03 ... 2.016e+03 2.017e+03

In [68]:
a = xr.open_dataset('da_my_ersst.nc').sst
a

<xarray.DataArray 'sst' (time: 1690, lev: 1, lat: 89, lon: 180)>
[27073800 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -88.0 -86.0 -84.0 -82.0 -80.0 ... 82.0 84.0 86.0 88.0
  * lev      (lev) float64 0.0
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) datetime64[ns] 1880-01-01 1880-02-01 ... 2020-10-01
Attributes:
    long_name:      Extended reconstructed sea surface temperature
    standard_name:  sea_surface_temperature
    units:          degree_C
    valid_min:      -3.0
    valid_max:      45.0

In [69]:
b = a[1600, 0, :, 5]

In [70]:
b

<xarray.DataArray 'sst' (lat: 89)>
array([      nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan, -1.8     , -1.8     , -1.8     ,
       -1.034459, -0.75012 , -0.540792, -0.267674,  0.060062,  0.763217,
        1.987814,  3.610058,  5.420043,  7.323426,  9.494114, 12.061518,
       14.749191, 16.956528, 18.319082, 18.960539, 19.244287, 19.467854,
       19.599054, 19.616737, 19.622509, 19.58602 , 19.646584, 20.310984,
       21.845987, 23.828941, 25.556774, 26.661932, 27.11139 , 26.90116 ,
       26.412651, 26.481514, 27.509047, 28.797558, 29.4703  ,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan, 17.902445, 17.713173, 17.486216,
       17.278526,       nan,       nan,       nan,       nan,  9.56851 ,
        9.633938,  9.762923,       nan,       nan,  8.215516,  7.978846,
        7.565221,  6.96719 ,  6.029084,  4.936018,  4.164358,  3.295709,
        1.582739, -1.8     , -1.8     , -1.8     , -1.8     ], dtype=float32)
Coordinates:
  * lat      (lat) float64 -88.0 -86.0 -84.0 -82.0 -80.0 ... 82.0 84.0 86.0 88.0
    lev      float64 0.0
    lon      float64 10.0
    time     datetime64[ns] 2013-05-01
Attributes:
    long_name:      Extended reconstructed sea surface temperature
    standard_name:  sea_surface_temperature
    units:          degree_C
    valid_min:      -3.0
    valid_max:      45.0

In [73]:
a = xr.open_dataset('da_o_globaltempv5.nc').anom

In [74]:
a

<xarray.DataArray 'anom' (time: 1689, z: 1, lat: 36, lon: 72)>
[4377888 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1880-01-01 1880-02-01 ... 2020-09-01
  * lat      (lat) float32 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
  * lon      (lon) float32 2.5 7.5 12.5 17.5 22.5 ... 342.5 347.5 352.5 357.5
  * z        (z) float32 0.0
Attributes:
    long_name:              Global Temperature Anomalies
    standard_name:          surface_temperature_anomaly
    coverage_content_type:  physicalMeasurement
    units:                  degrees C
    valid_min:              -40.0
    valid_max:              40.0

In [75]:
b = a[1600, 0, :, 5]

In [76]:
b

<xarray.DataArray 'anom' (lat: 36)>
array([      nan,       nan,       nan,       nan,       nan, -0.191629,
       -0.239836, -0.199724, -0.005309,  0.334719,  0.450888,  0.185744,
        0.591843,  0.438826,  0.351124,  0.406724,  0.413556,  0.245094,
        0.198506,  0.335659,  1.070012,  1.010701,  1.320988,  1.825311,
        1.643378,  2.229429,  2.974525,  2.931451,  3.477331,  3.616562,
        3.028609,  4.069273,  1.570916,  3.04    ,       nan,       nan],
      dtype=float32)
Coordinates:
    time     datetime64[ns] 2013-05-01
  * lat      (lat) float32 -87.5 -82.5 -77.5 -72.5 -67.5 ... 72.5 77.5 82.5 87.5
    lon      float32 27.5
    z        float32 0.0
Attributes:
    long_name:              Global Temperature Anomalies
    standard_name:          surface_temperature_anomaly
    coverage_content_type:  physicalMeasurement
    units:                  degrees C
    valid_min:              -40.0
    valid_max:              40.0

In [81]:
rf = np.loadtxt("da_o_sahelprecip19012017.txt", skiprows=8,)

In [89]:
np.argwhere(np.isnan(rf))


array([], shape=(0, 2), dtype=int64)

In [30]:
x = xr.DataArray([0, 7, 1, np.nan, 2], dims=["x"])

In [31]:
x.mean()

<xarray.DataArray ()>
array(2.5)